# Import Library & Set Dataset Name

In [4]:
import os
import warnings
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
warnings.simplefilter("ignore", pd.errors.DtypeWarning)

from preprocess import get_cellbender_data,Filter_QC,Filter_Doublet,Normalize,FindVariableGenes,Integrate
from plot import *
import scanpy as sc
from pacmap import LocalMAP

In [5]:
import rpy2.robjects as ro
from rpy2.robjects.conversion import localconverter
from R import get_converter,R_preload
# R_preload()
converter = get_converter()

In [8]:
%matplotlib inline
study = "paper_processed"
R_converted = False

DATADIR = "../../data"
REFDIR = './references'
DOUBLETMETHODS = ['scDblFinder','DoubletFinder','doubletdetection','scrublet']

# Access adata

In [ ]:
filetype = 'h5ad' if R_converted else 'h5'

basedir = os.path.join(DATADIR,'cellbender',study)
samps = os.listdir(basedir)
print(samps)
filt_files = [os.path.join(basedir,s,f'output_filtered.{filetype}') for s in samps]
adata = [get_cellbender_data(file,filetype) for file in filt_files]

for d in adata:
    d.obs['Experiment'], d.obs['Groups'], d.obs['Sample'] = np.array(d.obs['Identifier'].str.split('-').to_list()).T
    d.obs['Condition'], d.obs['Sample Type'] = np.array(d.obs['Groups'].str.split('_').to_list()).T

adata

## Data Structures

#### SingleCellExperiment object (in R) is presented below
![SingleCellExperiment Object (in R)](https://hbctraining.github.io/scRNA-seq/img/sce_description.png)

#### AnnData object (in Python) is similar!
* rowData is stored in `adata.vars`
* colData is stored in `adata.obs`
* reducedDims is stored in `adata.obsm`
* layers are stored in `adata.layers`

# Preprocessing

### QC

In [ ]:
adata = [Filter_QC(dat, mt_ratio=15, ribo_ratio=15) for dat in adata]
PlotQC = True
adata

#### QC Plots

Plot QC Filters for 1 dataset

In [ ]:
if PlotQC:
    # sns.displot(adata[1].obs['total_counts'], bins=100, kde=False).figure
    tmp = adata[0]
    sc.pl.violin(
        tmp,
        ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
        jitter=0.4,
        multi_panel=True
    )
    sc.pl.scatter(tmp, "total_counts", "n_genes_by_counts", color="pct_counts_mt",vmax=100)

Plot QC Metrics for all datasets

In [ ]:
def QC_Plot(df, value, groupby):
    sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

    g = sns.FacetGrid(df, row=groupby, hue=groupby, aspect=15, height=0.5, palette="tab20")

    g.map(sns.kdeplot, value, clip_on=False, fill=True, alpha=1, linewidth=1.5)
    g.map(sns.kdeplot, value, clip_on=False, color="w", lw=2)

    g.map(plt.axhline, y=0, lw=2, clip_on=False)

    def label(x, color, label):
        ax = plt.gca()
        ax.text(0, .2, label, fontweight="bold", color=color,
                ha="left", va="center", transform=ax.transAxes)

    g.map(label, value)

    g.figure.subplots_adjust(hspace=-.6)

    g.set_titles("")
    g.set(yticks=[], ylabel="")
    g.despine(bottom=True, left=True)

    for ax in g.axes.flat:
        ax.axvline(x=df[value].median(), color='r', linestyle='-')

    return g.figure

if PlotQC:
    df = pd.concat([x.obs for x in adata])
    groupby = 'Groups'
    df = df.sort_values(groupby)

    QC_Plot(df,"log1p_total_counts",groupby)
    QC_Plot(df,"pct_counts_mt",groupby)
    QC_Plot(df,"n_genes",groupby)
    QC_Plot(df,"pct_counts_in_top_20_genes",groupby)

### Doublets

In [ ]:
compare_doublets = False
method = 'doubletdetection'
method_short = 'dd'

if compare_doublets is True:
    for method in DOUBLETMETHODS:
        adata = [Filter_Doublet(dat,method,remove=False) for dat in adata]

else:
    print(f"Doublet Method: {method}")
    adata = [Filter_Doublet(dat,method,remove=True) for dat in adata]
    for dat in adata:
        print(len(dat), dat.uns['doublets_removed'])
        
adata

In [ ]:
# save
annotation = method_short
with open(os.path.join(DATADIR,'processed', study, 'py', '1_preprocessed', f'{annotation}.pickle'), 'wb') as handle:
    pickle.dump(adata, handle)

## Pre-Integration Cell Labels (deprecated)

### CellTypist

In [ ]:
import celltypist
from celltypist import models
models.download_models(force_update = False)
print(models.models_path)


In [ ]:
def Annotate_celltypist(adata, model, modelname):
    #celltypist-specific processing
    tmp = adata.copy()
    sc.pp.normalize_total(tmp,target_sum=1e4)
    sc.pp.log1p(tmp)

    predictions = celltypist.annotate(tmp, model=model, majority_voting=False).to_adata()
    adata.obs[f'{modelname}-label'] = predictions.obs.loc[adata.obs.index,"predicted_labels"]
    adata.obs[f'{modelname}-score'] = predictions.obs.loc[adata.obs.index,"conf_score"]
    return adata

model = models.Model.load(model="Immune_All_Low.pkl")
modelname = "CT_immune_highres"
adata = [Annotate_celltypist(dat,model,modelname) for dat in adata]

model = models.Model.load(model="Immune_All_High.pkl")
modelname = "CT_immune_lowres"
adata = [Annotate_celltypist(dat,model,modelname) for dat in adata]

Visualize

In [ ]:
tmp = adata[1].copy()
sc.pp.neighbors(tmp)
sc.tl.umap(tmp)

In [ ]:
sc.pl.umap(tmp,color=['CT_immune_highres-label','CT_immune_lowres-label'],legend_loc='on adata')
sns.histplot(tmp.obs['CT_immune_lowres-score'])
plt.figure()

f,ax = plt.subplots(2, 1)
_ = sns.histplot(tmp.obs['CT_immune_lowres-score'],ax=ax[0])
_ = sns.histplot(tmp.obs['CT_immune_highres-score'],ax=ax[1])

# Integration

### Integrate

In [ ]:
# load adata
annotation = "scDF"
with open(os.path.join(DATADIR,'processed',study,'py',
                                  '1_preprocessed', f'{annotation}.h5ad'),
                                  'rb') as handle:
    adata = pickle.load(handle)

In [ ]:
batch_column = 'Groups'
hvg_kind = 'seurat'
int_kind = 'harmony'

# merge
adata = sc.concat(adata, join='outer')
adata.raw = adata.copy()

# process & integrate
adata = Normalize(adata, kind='log1p', batch_column=batch_column)
adata = FindVariableGenes(adata, kind=hvg_kind, batch_column=batch_column)
adata = Integrate(adata, kind = int_kind, batch_column=batch_column, use_var_genes=False)

In [ ]:
# save
annotation = "scDF-seuratV3-harmony"
adata.write(os.path.join(DATADIR,'processed',study,'py',
                                  '2_integrated', f'{annotation}.h5ad'))

### Plot

In [ ]:
annotation = "scDF-seuratV3-harmony"
adata = sc.read_h5ad(os.path.join(DATADIR,'processed',study,'py',
                                  '2_integrated', f'{annotation}.h5ad'))

In [ ]:
# plot embeddings
f = plt.figure(figsize=(10,5), layout="constrained")
check_integration(adata, "Groups", f, mini=True)
f = plt.figure(figsize=(18,12), layout="constrained")
check_integration(adata, "Groups", f, nrow=2, ncol=2, mini=False)

In [ ]:
# Feature marker plots
markers = ["Adipoq", "Pdgfra", "Upk3b", "Cdh5", "Rgs5", "Adgre1", "Flt3", "Cpa3", "Skap1", "Igkc"]
sc.pl.umap(adata, color=markers, layer='normalized')
sc.pl.embedding(adata, basis="LocalMAP", color=markers, layer='normalized')

# Analysis

## Overall Clustering

### Cluster

In [ ]:
# load data
annotation = "scDF-seuratV3-harmony"
adata = sc.read_h5ad(os.path.join(DATADIR,'processed',study,'py',
                                  '2_integrated', f'{annotation}.h5ad'))
adata.obs["Groups"] = pd.Categorical(adata.obs["Groups"],categories=['LFD_eWAT', 'HFD_eWAT', 'LFD_iWAT', 'HFD_iWAT'], ordered=True)
adata.raw = adata.copy()

In [ ]:
resolution = 1
compareUMAP = True

# leiden clustering
sc.tl.leiden(adata, resolution,random_state=123)
print(f"{len(adata.obs['leiden'].unique())} clusters found at resolution {int(adata.uns['leiden']['params']['resolution'])}!")

if compareUMAP is True:
    try: 
        del adata.uns["leiden_colors"]
    except KeyError:
        pass

    f = plt.figure(figsize=(12,5), layout="constrained")
    check_integration(adata, "leiden", f, mini=True)

# figure prep
cluster_c = color_gen(adata.obs["leiden"])
f = plt.figure(figsize=(25,15),layout="constrained")
sf = f.subfigures(1,2)

# Large LocalMAP plot
axs = sf[0].subplots(4,2)
gs = axs[0,0].get_gridspec()
empty_axs(axs)
ax = sf[0].add_subplot(gs[:3,:])
sc.pl.embedding(adata, basis='LocalMAP', color=['leiden'], ax=ax, show=False, legend_loc='on data', legend_fontoutline=1, legend_fontsize=15, palette=cluster_c)
ax = sf[0].add_subplot(gs[3,0])
sc.pl.embedding(adata, basis='LocalMAP', color=['Groups'], ax=ax, show=False, alpha=0.7)
ax = sf[0].add_subplot(gs[3,1])
sc.pl.embedding(adata, basis='LocalMAP', color=['Condition'], ax=ax, show=False, alpha=0.7)

# Violin marker plots
markers = ["Adipoq", "Pdgfra", "Upk3b", "Cdh5", "Rgs5", "Adgre1", "Flt3", "Cpa3", "Skap1", "Igkc"]
cluster_violinplot(adata, markers, "leiden", sf[1])

### Doublet Checking
only use on '-hasDoublets' verions!

In [ ]:
sc.pl.embedding(adata, basis='LocalMAP', color = [f"predicted_doublet-{method}" for method in DOUBLETMETHODS],ncols=2)
sc.pl.embedding(adata, basis='LocalMAP', color = 'n_genes')
for method in DOUBLETMETHODS:
    print(adata.obs.groupby('leiden')[f'predicted_doublet-{method}'].mean().sort_values())

In [ ]:
# sc.tl.rank_genes_groups(adata, groupby='leiden')
gene_groups = sc.get.rank_genes_groups_df(adata, group=None)
gene_groups[gene_groups.group == '0'].head(20)

### Annotate

In [ ]:
remap = False

if remap is True:
    mapping = {
        0  : 'Adipocyte',  1  : 'Fibroblast',  2  : 'Fibroblast',   3  : 'Adipocyte',  4  : 'Macrophage',
        5  : 'Fibroblast',   6  : 'Adipocyte',   7  : 'Adipocyte',  8  : 'Macrophage', 9  : 'Adipocyte',
        10 : 'Adipocyte',   11 : 'Endothelial',   12 : 'Endothelial',   13 : 'Fibroblast',   14 : 'Mesothelial',
        15 : 'Macrophage',  16 : 'PC/SMC',   17 : 'Adipocyte',   18 : 'Adipocyte',   19 : 'Dendritic Cell',
        20 : 'Macrophage',  21 : 'Fibroblast',  22 : 'T cell',  23 : 'Macrophage',
    }

    try:
        assert len(mapping) == len(adata.obs['leiden'].unique())
    except AssertionError:
        print(f"At resolution {resolution}, {len(adata.obs['leiden'].unique())} clusters identified, but only {len(mapping)} mappings provided!")

    adata.obs['cell_type'] = adata.obs['leiden'].astype(int).replace(mapping)
    adata.obs['cell_type'] = pd.Categorical(adata.obs['cell_type'],
                                            categories=['Adipocyte', 'Fibroblast', 'Mesothelial', 'Endothelial', 'PC/SMC', 'Macrophage', 'Dendritic Cell', 'T cell'],
                                            ordered=True)

for obsm in ["X_umap", "LocalMAP"]:
    f,axs = plt.subplots(1,3,figsize=(20,5),layout='constrained')
    sc.pl.embedding(adata, basis=obsm, color=['leiden'], ax=axs[0], show=False, palette=color_gen(adata.obs["leiden"]))
    sc.pl.embedding(adata, basis=obsm, color = ['cell_type'], ax=axs[1], show=False, palette=color_gen(adata.obs['cell_type']))
    sc.pl.embedding(adata, basis=obsm, color = ['Groups'], ax=axs[2], show=False, palette=color_gen(adata.obs['Groups']), alpha=0.5)

f = plt.figure(figsize=(8,12),layout="constrained")
markers = ["Adipoq", "Pdgfra", "Upk3b", "Cdh5", "Rgs5", "Adgre1", "Flt3", "Cpa3", "Skap1", "Igkc"]
cluster_violinplot(adata, markers, "cell_type", f)

In [ ]:
f = plt.figure(figsize=(10, 5), layout="constrained")
axs = f.subplots(1, 2)
cluster_stackedbarplot(adata, "Groups", "cell_type", pct=False, ax=axs[0])
cluster_stackedbarplot(adata, "Groups", "cell_type", pct=True, ax=axs[1])

f = plt.figure(figsize=(10, 5), layout="constrained")
axs = f.subplots(1, 2)
cluster_stackedbarplot(adata, "cell_type", "Groups", pct=False, ax=axs[0])
cluster_stackedbarplot(adata, "cell_type", "Groups", pct=True, ax=axs[1])

### DEGs

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='cell_type',
                        use_raw=False, layer="normalized", key_added='de_all',
                        method='wilcoxon')

f, ax = plt.subplots(1,1,figsize=(30,5),layout="constrained")
sc.pl.rank_genes_groups_dotplot(
    adata, groupby="cell_type", key='de_all',
    standard_scale="var", n_genes=10,
    var_group_rotation=30, ax=ax
)

comparison = ["Adipocyte", "Fibroblast", "Macrophage"]
f, ax = plt.subplots(1,1,figsize=(30,5),layout="constrained")
sc.pl.rank_genes_groups_dotplot(
    adata, groupby="cell_type", groups=comparison, key='de_all',
    standard_scale="var", n_genes=30,
    var_group_rotation=30, ax=ax
)

df = sc.get.rank_genes_groups_df(adata, group=None, key='de_all')

In [ ]:
marker_of_interest = "Adgre1"
df[df["names"] == marker_of_interest]

celltype_of_interest = "Adipocyte"
df[df["group"] == celltype_of_interest].head(50)

In [ ]:
# save
annotation = "scDF-seuratV3-harmony-leiden_1.0"
adata.write(os.path.join(DATADIR,'processed',study,'py',
                                  '3_annotated', f'{annotation}.h5ad'))

## Subclustering

In [ ]:
# load
annotation = "scDF-seuratV3-harmony-leiden_1.0"
adata = sc.read_h5ad(os.path.join(DATADIR,'processed',study,'py',
                                  '3_annotated', f'{annotation}.h5ad'))

In [ ]:
# subset
adata_macros = adata[adata.obs["cell_type"] == "Macrophage"].copy()
del adata_macros.uns, adata_macros.varm, adata_macros.obsp
sc.pp.filter_cells(adata_macros, min_genes=200)
sc.pp.filter_genes(adata_macros, min_cells=5)

# UMAP
sc.pp.neighbors(adata_macros, use_rep = "integrated")
sc.tl.umap(adata_macros, key_added='umap_macro')

#LocalMAP
lm = LocalMAP()
adata_macros.obsm["LocalMAP_macro"] = lm.fit_transform(adata_macros.obsm['integrated'])

In [ ]:
# integration check
f = plt.figure(figsize=(18,12), layout="constrained")
check_integration(adata_macros, "Groups", f, embeddings=["umap_macro", "LocalMAP_macro"], nrow=2, ncol=2)

In [ ]:
# cluster
sc.tl.leiden(adata_macros, resolution=0.5, key_added='leiden_macro')

order_obs(adata_macros,"leiden_macro",pd.Series([2,1,6,0,4,3,7,5]).astype(str))

# figure prep
cluster_c = color_gen(adata_macros.obs["leiden_macro"])
f = plt.figure(figsize=(25,15),layout="constrained")
sf = f.subfigures(1,2)

axs = sf[0].subplots(4,2)
gs = axs[0,0].get_gridspec()
empty_axs(axs)

# Large LocalMAP plot
embedding = "umap_macro"
ax = sf[0].add_subplot(gs[:3,:])
sc.pl.embedding(adata_macros, basis=embedding, color=['leiden_macro'], ax=ax, show=False, legend_loc='on data', legend_fontoutline=2, legend_fontsize=20, palette=cluster_c)
ax = sf[0].add_subplot(gs[3,0])
sc.pl.embedding(adata_macros, basis=embedding, color=['Groups'], ax=ax, show=False, alpha=0.7)
ax = sf[0].add_subplot(gs[3,1])
sc.pl.embedding(adata_macros, basis=embedding, color=['Condition'], ax=ax, show=False, alpha=0.7)

# Violin marker plots
markers = ["Adgre1", "Cd209f", "Creb5", "Gpnmb", "Cd226", "Plac8", "Slpi", "Col1a2"]
cluster_violinplot(adata_macros, markers, "leiden_macro", sf[1])

In [ ]:
# reannotate
adata_macros.obs["leiden_macro_group"] = adata_macros.obs["leiden_macro"].map(
    {'2' : 1,
     '1' : 2, '6' : 3,
     '0' : 3, '4' : 3, '3' : 3,
     '7' : 4,
     '5' : 5,
     }).astype(str)
order_obs(adata_macros,"leiden_macro_group",pd.Series(np.arange(1,6)).astype(str))

f = plt.figure(figsize=(10, 5), layout="constrained")
axs = f.subplots(1, 2)
cluster_stackedbarplot(adata_macros, "Groups", "leiden_macro_group", pct=False, ax=axs[0])
cluster_stackedbarplot(adata_macros, "Groups", "leiden_macro_group", pct=True, ax=axs[1])

f = plt.figure(figsize=(10, 5), layout="constrained")
axs = f.subplots(1, 2)
cluster_stackedbarplot(adata_macros, "leiden_macro_group", "Groups", pct=False, ax=axs[0])
cluster_stackedbarplot(adata_macros, "leiden_macro_group", "Groups", pct=True, ax=axs[1])

In [ ]:
# find DEGs
sc.tl.rank_genes_groups(adata_macros, groupby='leiden_macro_group', key_added="de_macro",
                        use_raw=False, layer="normalized", 
                        method='wilcoxon')

# del adata_macros.uns['dendrogram_leiden_macro'], adata_macros.uns['dendrogram_leiden_macro_group']
f, ax = plt.subplots(1,1,figsize=(15,5),layout="constrained")
sc.pl.rank_genes_groups_dotplot(
    adata_macros, groupby="leiden_macro_group", key='de_macro',
    standard_scale="var", n_genes=10, min_logfoldchange=2, ax=ax
)

df = sc.get.rank_genes_groups_df(adata_macros, group=None, key="de_macro")

In [ ]:
# save
annotation = "scDF-seuratV3-harmony-leiden_1.0-macro_leiden_0.5"
adata.write(os.path.join(DATADIR,'processed',study,'py',
                                  '4.0_subclustered', f'{annotation}.h5ad'))

## Compositional Analysis
* scCODA (clusters) - https://www.sc-best-practices.org/conditions/compositional.html#with-labeled-clusters
* tascCODA (hierachical clusters) - https://www.sc-best-practices.org/conditions/compositional.html#with-labeled-clusters-and-hierarchical-structure
* scanpro (reps, clusters) https://scanpro.readthedocs.io/en/latest/proportion_analysis.html
* Milo (none) https://www.sc-best-practices.org/conditions/compositional.html#without-labeled-clusters

## CCC

In [9]:
from ccc import *
from liana.method import cellphonedb, rank_aggregate

annotation = "scDF-seuratV3-harmony-leiden_1.0"
adata = sc.read_h5ad(os.path.join(DATADIR,'processed',study,'py',
                                  '3_annotated', f'{annotation}.h5ad'))

### Liana Approach

In [10]:
resource_opts = ['baccin2019', 'cellcall', 'cellchatdb', 'cellinker', 'cellphonedbv5', 'cellphonedb', 'celltalkdb', 'connectomedb2020', 'consensus', 'embrace', 'guide2pharma', 'hpmr', 'icellnet', 'italk', 'kirouac2010', 'lrdb', 'mouseconsensus', 'ramilowski2015']
resources = liana_mouse_resource(resource_opts)

ccc_db = merge_dbs(resources).reset_index(drop=True)

converting resource baccin2019 from human to mouse
converting resource cellcall from human to mouse
converting resource cellchatdb from human to mouse
converting resource cellinker from human to mouse
converting resource cellphonedbv5 from human to mouse
converting resource cellphonedb from human to mouse
converting resource celltalkdb from human to mouse
converting resource connectomedb2020 from human to mouse
converting resource consensus from human to mouse
converting resource embrace from human to mouse
converting resource guide2pharma from human to mouse
converting resource hpmr from human to mouse
converting resource icellnet from human to mouse
converting resource italk from human to mouse
converting resource kirouac2010 from human to mouse
converting resource lrdb from human to mouse
found lowercase letters in first cell 'Dll1' of mouseconsensus, not converting!
converting resource ramilowski2015 from human to mouse


In [11]:
# cellphoneDB only
cellphonedb(
    adata, groupby="cell_type", layer='normalized', use_raw=False, key_added="ccc_cellphoneDB",
    return_all_lrs=True, verbose=True, n_jobs=10, resource=ccc_db[["ligand", "receptor"]]
)
adata.uns["ccc_cellphoneDB"] = adata.uns["ccc_cellphoneDB"].merge(ccc_db[['ligand', 'receptor', 'db_sources']],
                                                                  left_on=["ligand_complex", "receptor_complex"],
                                                                  right_on=['ligand','receptor'],
                                                                  how='left')
adata.uns["ccc_cellphoneDB"]

Using the `normalized` layer!


Using provided `resource`.


0.25 of entities in the resource are missing from the data.


Generating ligand-receptor stats for 13861 samples and 1529 features


100%|██████████| 1000/1000 [00:48<00:00, 20.75it/s]


,ligand_x,ligand_complex,ligand_means,ligand_props,receptor_x,receptor_complex,receptor_means,receptor_props,source,target,lrs_to_keep,lr_means,cellphone_pvals,ligand_y,receptor_y,db_sources
0,Col1a2,Col1a2,3.227245,0.989616,Cd36,Cd36,2.691967,0.954268,Fibroblast,PC/SMC,True,2.959606,0.0,Col1a2,Cd36,"baccin2019, cellinker, celltalkdb, consensus, ..."
1,Col1a2,Col1a2,3.227245,0.989616,Cd36,Cd36,2.668688,0.872701,Fibroblast,Macrophage,True,2.947967,0.0,Col1a2,Cd36,"baccin2019, cellinker, celltalkdb, consensus, ..."
2,Col3a1,Col3a1,3.611327,0.972712,Itga1,Itga1_Itgb1,2.257015,0.987805,Fibroblast,PC/SMC,True,2.934171,0.0,Col3a1,Itga1_Itgb1,"cellphonedb, cellphonedbv5, consensus, icellne..."
3,Dcn,Dcn,3.699373,0.995170,Egfr,Egfr,2.088059,0.870321,Fibroblast,Fibroblast,True,2.893716,0.0,Dcn,Egfr,"baccin2019, cellinker, celltalkdb, connectomed..."
4,Col1a2,Col1a2,3.227245,0.989616,Cd36,Cd36,2.522447,0.897561,Fibroblast,Endothelial,True,2.874846,0.0,Col1a2,Cd36,"baccin2019, cellinker, celltalkdb, consensus, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319547,Guca1b,Guca1b,0.000366,0.003049,Gucy2f,Gucy2f,0.000000,0.000000,PC/SMC,Mesothelial,False,0.056011,1.0,Guca1b,Gucy2f,guide2pharma
319548,Guca1a,Guca1a,0.004512,0.009146,Gucy2f,Gucy2f,0.000000,0.000000,PC/SMC,Mesothelial,False,0.056011,1.0,Guca1a,Gucy2f,guide2pharma
319549,Gdf9,Gdf9,0.000000,0.000000,Acvr1c,Acvr1c,0.009380,0.005917,PC/SMC,Mesothelial,False,0.056011,1.0,Gdf9,Acvr1c,guide2pharma
319550,Gdf9,Gdf9,0.000000,0.000000,Acvr2b,Acvr2b,0.030437,0.041420,PC/SMC,Mesothelial,False,0.056011,1.0,Gdf9,Acvr2b,guide2pharma


In [12]:
# Consensus
li.method.show_methods()

rank_aggregate(
    adata, groupby="cell_type", layer='normalized', use_raw=False, key_added="ccc_aggregate",
    return_all_lrs=True, verbose=True, n_jobs=10, resource=ccc_db[["ligand", "receptor"]]
)
adata.uns["ccc_aggregate"] = adata.uns["ccc_aggregate"].merge(ccc_db[['ligand', 'receptor', 'db_sources']],
                                                              left_on=["ligand_complex", "receptor_complex"],
                                                              right_on=['ligand','receptor'],
                                                              how='left')
adata.uns["ccc_aggregate"]

Using the `normalized` layer!


Using provided `resource`.


0.25 of entities in the resource are missing from the data.


Generating ligand-receptor stats for 13861 samples and 1529 features
Assuming that counts were `natural` log-normalized!
Running CellPhoneDB


100%|██████████| 1000/1000 [00:05<00:00, 184.47it/s]


Running Connectome
Running log2FC
Running NATMI
Running SingleCellSignalR


,source,target,ligand_complex,receptor_complex,lr_means,cellphone_pvals,expr_prod,scaled_weight,lr_logfc,spec_weight,lrscore,specificity_rank,magnitude_rank,ligand,receptor,db_sources
0,Fibroblast,PC/SMC,Col1a2,Cd36,2.959606,0.0,8.687635,1.031312,2.414536,0.097880,0.969586,0.000132,2.647951e-10,Col1a2,Cd36,"baccin2019, cellinker, celltalkdb, consensus, ..."
1,Fibroblast,Macrophage,Col1a2,Cd36,2.947967,0.0,8.612510,1.023153,2.770974,0.097034,0.969458,0.000132,1.059176e-09,Col1a2,Cd36,"baccin2019, cellinker, celltalkdb, consensus, ..."
2,Fibroblast,PC/SMC,Col3a1,Itga1_Itgb1,2.934171,0.0,8.150819,2.075122,3.615517,0.171754,0.968631,0.000016,2.383138e-09,Col3a1,Itga1_Itgb1,"cellphonedb, cellphonedbv5, consensus, icellne..."
3,Fibroblast,Endothelial,Col1a2,Cd36,2.874846,0.0,8.140555,0.971899,2.450606,0.091716,0.968612,0.000132,6.619781e-09,Col1a2,Cd36,"baccin2019, cellinker, celltalkdb, consensus, ..."
4,Fibroblast,Fibroblast,Dcn,Egfr,2.893716,0.0,7.724507,1.149170,3.296879,0.184209,0.967805,0.000132,6.619781e-09,Dcn,Egfr,"baccin2019, cellinker, celltalkdb, connectomed..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319547,T cell,T cell,Wnt9a,Lrp5,0.056011,1.0,0.003128,-0.866478,-2.660542,0.000220,0.376919,1.000000,1.000000e+00,Wnt9a,Lrp5,"cellcall, cellinker"
319548,T cell,T cell,Wnt9a,Lrp6,0.056011,1.0,0.003128,-0.866478,-2.660542,0.000220,0.376919,1.000000,1.000000e+00,Wnt9a,Lrp6,"cellcall, cellinker"
319549,T cell,T cell,Wnt9a,Sfrp1,0.056011,1.0,0.003128,-0.866478,-2.660542,0.000220,0.376919,1.000000,1.000000e+00,Wnt9a,Sfrp1,cellphonedbv5
319550,T cell,T cell,Zpbp2,Cd80,0.056011,1.0,0.003128,-0.866478,-2.660542,0.000220,0.376919,1.000000,1.000000e+00,Zpbp2,Cd80,cellinker


In [13]:
# save
annotation = "scDF-seuratV3-harmony-leiden_1.0"
adata.write(os.path.join(DATADIR,'processed',study,'py',
                                  'cell_communication', f'{annotation}.h5ad'))

### Non-Liana

In [ ]:
### NON LIANA APPROACH
R_preload()
with localconverter(converter):
    ro.globalenv["mouse_genes"] = adata.var.index
    ro.r("""
    # see https://neurogenomics.github.io/orthogene/articles/orthogene.html
    genes <- orthogene::convert_orthologs(gene_df = mouse_genes,
                                            gene_output = "dict",
                                            input_species = "mouse",
                                            output_species = "human",
                                            non121_strategy = "drop_input_species",
                                            method = "gprofiler") 
    mappings <- data.frame(genes,row.names=names(genes))
    """)
    mappings = ro.globalenv["mappings"].to_dict()['genes']

converted = tmp_adata[:,tmp_adata.var_names.isin([m for m in mappings])].copy()
converted.var_names = converted.var_names.map(mappings)

# Other DEGs Attempts
* psuedobulking - requires replicates (https://hbctraining.github.io/scRNA-seq/lessons/pseudobulk_DESeq2_scrnaseq.html)
    * [R] edgeR
    * decoupleR - https://decoupler-py.readthedocs.io/en/latest/notebooks/scell/rna_psbk.html
* single cell DEGS
    * MAST for single cell DEGS
    * Scanpy's Wilcoxon test 

In [ ]:
f = plt.figure(figsize=(30,5),layout="constrained")
ax = f.subplots(1,1)
sc.pl.rank_genes_groups_dotplot(
    adata_macros, groupby="leiden_macro",
    standard_scale="var", n_genes=10, key="rank_genes_groups_macro",
    var_group_rotation=30, ax=ax, min_logfoldchange=2,
)

In [ ]:
# load
study = 'paper_processed'
annotation = "-scDF-seuratV3-harmony-leiden1.0"
adata = sc.read_h5ad(os.path.join(DATADIR,'processed',study,f'py-annotated{annotation}.h5ad'))
%load_ext rpy2.ipython
R_preload()

#### MAST

In [ ]:
adata2 = adata.copy()
adata2.X = adata2.layers["normalized"].copy()

df = pd.DataFrame.sparse.from_spmatrix(data=adata2.X, index=adata2.obs_names, columns=adata2.var_names)
df.join(adata2.obs)
adata_tmp = sc.AnnData(df[adata2.var_names], obs=df.drop(columns=adata2.var_names))

# def prep_anndata(adata_):
#     def fix_dtypes(adata_):
#         for i in [adata_.X, adata_.obs_names, adata_.var_names]:
#             print(i.shape)
#         df = pd.DataFrame(data=adata_.X)#, index=adata_.obs_names, columns=adata_.var_names)
#         df = df.join(adata_.obs)
#         print(df.head())
#         return sc.AnnData(df[adata_.var_names], obs=df.drop(columns=adata_.var_names))

#     adata_ = fix_dtypes(adata_)
#     sc.pp.filter_genes(adata_, min_cells=3)
#     return adata_

# adata_prep = prep_anndata(adata2)
# adata_prep

In [ ]:
adata_tmp

#### PseudoBulking

##### Attempt 2

In [ ]:
with localconverter(converter):
    ro.globalenv['sce'] = adata

In [ ]:
%%R
seurat <- as.Seurat(sce, data = NULL)
seurat
# counts <- GetAssayData(object = seurat, slot = "counts", assay = "RNA")

##### Attempt 1

In [ ]:
pb_adatas = []
for cell_type in adata.obs['cell_type'].unique():
    cell_pop = adata[adata.obs["cell_type"] == cell_type].copy()
    pb_cell_pop = sc.get.aggregate(cell_pop,by='Groups',func='sum',layer='counts')
    pb_adata = sc.AnnData(
        X=pb_cell_pop.layers['sum'],
        obs=pb_cell_pop.obs,
        var=pb_cell_pop.var.copy()
    )
    pb_adata.obs['cell_type'] = cell_type
    pb_adatas.append(pb_adata)
pb = sc.concat(pb_adatas)
pb.obs[['Condition', 'Sample Type']] = pb.obs['Groups'].str.split('_', expand=True)

counts_matrix = pb.X.T
if hasattr(counts_matrix, 'toarray'):
    counts_matrix = counts_matrix.toarray()

print("pb.obs shape:", pb.obs.shape)
print("pb.obs columns:", pb.obs.columns.tolist())
print("pb.obs head:")
print(pb.obs.head())
print("pb.X shape:", pb.X.shape)

In [ ]:
with localconverter(converter):
    ro.globalenv['counts'] = pd.DataFrame(counts_matrix)
    ro.globalenv['metadata'] = pb.obs
    ro.globalenv['gene_names'] = pb.var_names.tolist()
    ro.r("""
        library(edgeR)
        library(dplyr)
        
        counts <- data.frame(counts)
        # Set up count matrix with proper row/column names
        rownames(counts) <- gene_names
        colnames(counts) <- paste(metadata$cell_type, metadata$Groups, sep="_")
        
        # Create sample metadata
        samples <- data.frame(metadata, stringsAsFactors = FALSE)
         
        rownames(samples) <- colnames(counts)
    """)

In [ ]:
%%R
options(max.print = 300)
cell_type_name = "Macrophage"
ct_samples <- samples[samples$cell_type == cell_type_name, ]
ct_counts <- counts[, rownames(ct_samples), drop=FALSE]
dge <- DGEList(counts = ct_counts, group = rownames(ct_samples))
dge$samples$Condition <- ct_samples$Condition
dge$samples$Sample.Type <- ct_samples$Sample.Type

# Filter low expressed genes
keep <- filterByExpr(dge)
dge <- dge[keep,,keep.lib.sizes=FALSE]
dge <- calcNormFactors(dge)

In [ ]:
%%R
# Create design matrix
# design <- model.matrix(~ Condition + Sample.Type, data = dge$samples)

# # Estimate dispersions
# dge <- estimateDisp(dge, design)

# # Fit GLM
# fit <- glmFit(dge, design)

model.matrix(~0 + group)

In [ ]:
%%R
lrt_condition <- glmLRT(fit, coef = "ConditionLFD")
topTags(lrt_condition)

In [ ]:
%%R
# Custom contrasts:
# contrasts <- makeContrasts(
#     HFD_vs_LFD_in_eWAT = HFD_eWAT - LFD_eWAT,
#     HFD_vs_LFD_in_iWAT = HFD_iWAT - LFD_iWAT, 
#     eWAT_vs_iWAT_in_HFD = HFD_eWAT - HFD_iWAT,
#     eWAT_vs_iWAT_in_LFD = LFD_eWAT - LFD_iWAT,
#     levels = design
# )


# lrt_condition <- glmLRT(fit)
# lrt <- glmLRT(fit, contrast = contrasts[,"HFD_vs_LFD_in_eWAT"])
# lrt_condition <- glmLRT(fit, coef = "conditionLFD")           # Main effect of diet
# lrt_tissue <- glmLRT(fit, coef = "sample_typeiWAT")           # Main effect of tissue  
# lrt_interaction <- glmLRT(fit, coef = "conditionLFD:sample_typeiWAT"
colnames(design)

In [ ]:
with localconverter(converter):
    ro.globalenv['counts'] = pd.DataFrame(counts_matrix)
    ro.globalenv['metadata'] = pb.obs
    ro.globalenv['gene_names'] = pb.var_names.tolist()
    ro.r("""
        library(edgeR)
        library(dplyr)
        
        counts <- data.frame(counts)
        # Set up count matrix with proper row/column names
        rownames(counts) <- gene_names
        colnames(counts) <- paste(metadata$cell_type, metadata$Groups, sep="_")
        
        # Create sample metadata
        samples <- data.frame(metadata, stringsAsFactors = FALSE)
         
        rownames(samples) <- colnames(counts)
        
        # Function to run EdgeR analysis for each cell type
        run_edger_analysis <- function(cell_type_name) {
            # Subset to specific cell type
            ct_samples <- rownames(samples[samples$cell_type == cell_type_name, ])
            ct_counts <- counts[, ct_samples, drop=FALSE]
            
            # Create DGEList
            dge <- DGEList(counts = ct_counts, samples = ct_samples)
            
            # Filter low expressed genes
            keep <- filterByExpr(dge, group = dge$samples$samples)
            dge <- dge[keep, ]
            
            # Calculate normalization factors
            dge <- calcNormFactors(dge)
            
            # Create design matrix
            design <- model.matrix(~ condition + sample_type, data = dge$samples)
            
            # Estimate dispersions
            dge <- estimateDisp(dge, design)
            
            # Fit GLM
            fit <- glmFit(dge, design)
            
            # Test for condition effect (assuming condition is what you want to test)
            lrt <- glmLRT(fit, coef = "conditionLFD")  # or whichever condition is reference
            
            # Get results
            results <- topTags(lrt, n = Inf)
            results_df <- as.data.frame(results$table)
            results_df$gene <- rownames(results_df)
            results_df$cell_type <- cell_type_name
            
            return(list(
                results = results_df,
                dge = dge,
                fit = fit,
                lrt = lrt
            ))
        }
        
        # Run analysis for each cell type
        cell_types <- unique(samples$cell_type)
        edger_results <- list()
        
        for(ct in cell_types) {
            cat("Analyzing cell type:", ct, "\n")
            edger_results[[ct]] <- run_edger_analysis(ct)
        }
        
        # Combine all results
        all_results <- do.call(rbind, lapply(edger_results, function(x) x$results))
        
        # Add FDR correction across all tests
        all_results$FDR_global <- p.adjust(all_results$PValue, method = "BH")
        
        print("Analysis complete!")
        print(paste("Total DE genes (FDR < 0.05):", sum(all_results$FDR < 0.05)))
        print(paste("Total DE genes (global FDR < 0.05):", sum(all_results$FDR_global < 0.05)))
    """)

# R space

In [ ]:
ro.r("""







""")

# Archive

### CellphoneDB v5 ONLY
* Get Database: https://github.com/ventolab/CellphoneDB/blob/master/notebooks/T0_DownloadDB.ipynb
* Prep Data: https://github.com/ventolab/CellphoneDB/blob/master/notebooks/DEGs_calculation/0_prepare_your_data_from_anndata.ipynb
* Run CPDB w/ statistical analysis: https://github.com/ventolab/CellphoneDB/blob/master/notebooks/T1_Method2.ipynb


In [ ]:
tmp_adata = sc.AnnData(X=adata.layers["normalized"].copy(),
                       obs=adata.obs[["cell_type","Groups","Condition","Sample Type", "Identifier"]].copy(),
                       var=pd.DataFrame(index=adata.var.index.copy()))

# load
savedir = os.path.join(DATADIR,'processed',study,'py','cellphonedb', annotation)

# metadata
print(converted.obs['cell_type'].values.describe())
df_meta = pd.DataFrame(data={'Cell':list(converted.obs.index),
                             'cell_type':[ i for i in converted.obs['cell_type']]
                            })
df_meta.set_index('Cell', inplace=True)

if df_meta.columns.isin(["barcode_sample"])[0]:
    assert np.all(converted.obs.index.sort_values() == df_meta["barcode_sample"].sort_values())
else:
    assert np.all(converted.obs.index.sort_values() == df_meta.index.sort_values())

# DEGs
DEGs = sc.get.rank_genes_groups_df(adata, group=None, key='de_all')
cond1 = DEGs['pvals_adj'] < 0.05 
cond2 = DEGs['logfoldchanges'] > 2
mask = [all(tup) for tup in zip(cond1, cond2)]
fDEGs = DEGs[mask]
fDEGs.columns = ["cluster", "gene"] + fDEGs.columns[2:].to_list()

# save relevant info to folder
converted.write(os.path.join(savedir,'counts.h5ad'))
df_meta.to_csv(os.path.join(savedir,'metadata.tsv'), sep = '\t')
fDEGs.to_csv(os.path.join(savedir,'DEGs.tsv'), index=False, sep='\t')

In [ ]:
from cellphonedb.utils import db_utils
from cellphonedb.src.core.methods import cpdb_analysis_method

version = 'v5.0.0'
cpdb_dir = os.path.join(REFDIR,f'cellphoneDB-{version}')
if not os.path.exists(cpdb_dir):
    print(f"Database not found! Downloading to {cpdb_dir}")
    db_utils.download_database(cpdb_dir, version)

cpdb_results = cpdb_analysis_method.call(
    cpdb_file_path = os.path.join(cpdb_dir,"cellphonedb.zip"),          # mandatory: CellphoneDB database zip file.
    meta_file_path = os.path.join(savedir,'metadata.tsv'),              # mandatory: tsv file defining barcodes to cell label.
    counts_file_path = converted,             # mandatory: normalized count matrix - a path to the counts file, or an in-memory AnnData object
    counts_data = 'hgnc_symbol',                                        # defines the gene annotation in counts matrix.
    # microenvs_file_path = microenvs_file_path,                        # optional (default: None): defines cells per microenvironment.
    score_interactions = True,                                          # optional: whether to score interactions or not. 
    output_path = os.path.join(savedir,'output.cpdb'),                  # Path to save results    microenvs_file_path = None,
    separator = '|',                                                    # Sets the string to employ to separate cells in the results dataframes "cellA|CellB".
    threads = 10,                                                       # number of threads to use in the analysis.
    threshold = 0.1,                                                    # defines the min % of cells expressing a gene for this to be employed in the analysis.
    result_precision = 3,                                               # Sets the rounding for the mean values in significan_means.
    debug = False,                                                      # Saves all intermediate tables emplyed during the analysis in pkl format.
    output_suffix = None                                                # Replaces the timestamp in the output files by a user defined string in the  (default: None)
)

In [ ]:
cpdb_results['interaction_scores']